In [ ]:
!pip install python-dotenv
!pip install requests

In [12]:
import json

def pprint(obj):
    print(json.dumps(obj, indent=2))

In [3]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
base_url = "https://api.openai.com"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [4]:
url = f"{base_url}/v1/moderations"

def get_moderation(text, model="omni-moderation-latest"):
    data = {
        "model": model,
        "input": text
    }
    return requests.post(url, headers=headers, json=data).json()

In [10]:
pprint(get_moderation("koreans are helpful"))

{
  "id": "modr-f390ee70b4450317a92e0505917fcaf5",
  "model": "omni-moderation-latest",
  "results": [
    {
      "flagged": false,
      "categories": {
        "harassment": false,
        "harassment/threatening": false,
        "sexual": false,
        "hate": false,
        "hate/threatening": false,
        "illicit": false,
        "illicit/violent": false,
        "self-harm/intent": false,
        "self-harm/instructions": false,
        "self-harm": false,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false
      },
      "category_scores": {
        "harassment": 1.8235460794662427e-05,
        "harassment/threatening": 4.495181578513688e-07,
        "sexual": 1.7778551863194697e-06,
        "hate": 1.2029639084557005e-06,
        "hate/threatening": 9.721546189381599e-08,
        "illicit": 4.710908830437064e-07,
        "illicit/violent": 1.4144759985825914e-07,
        "self-harm/intent": 4.495181578513688e-07,
        "self-harm/

In [14]:
pprint(get_moderation("koreans are racist"))

{
  "id": "modr-213dec6085e72784f6eef02ce10d5701",
  "model": "omni-moderation-latest",
  "results": [
    {
      "flagged": true,
      "categories": {
        "harassment": true,
        "harassment/threatening": false,
        "sexual": false,
        "hate": true,
        "hate/threatening": false,
        "illicit": false,
        "illicit/violent": false,
        "self-harm/intent": false,
        "self-harm/instructions": false,
        "self-harm": false,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false
      },
      "category_scores": {
        "harassment": 0.8110438458023302,
        "harassment/threatening": 0.0007820048173500451,
        "sexual": 1.8814207550495463e-05,
        "hate": 0.8277783843481699,
        "hate/threatening": 6.615174228741224e-05,
        "illicit": 1.2632042438707805e-05,
        "illicit/violent": 1.7778551863194697e-06,
        "self-harm/intent": 2.4300240743279605e-06,
        "self-harm/instructi

In [15]:
pprint(get_moderation("chess players are racist"))

{
  "id": "modr-542bd491dcdb9cb5b32cf9e91779b3f6",
  "model": "omni-moderation-latest",
  "results": [
    {
      "flagged": false,
      "categories": {
        "harassment": false,
        "harassment/threatening": false,
        "sexual": false,
        "hate": false,
        "hate/threatening": false,
        "illicit": false,
        "illicit/violent": false,
        "self-harm/intent": false,
        "self-harm/instructions": false,
        "self-harm": false,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false
      },
      "category_scores": {
        "harassment": 0.34248637740007,
        "harassment/threatening": 3.0534893211185946e-05,
        "sexual": 6.205049602300744e-06,
        "hate": 0.18323056495669537,
        "hate/threatening": 9.915273115920183e-06,
        "illicit": 1.971694219622292e-05,
        "illicit/violent": 2.2827997192887966e-06,
        "self-harm/intent": 6.339210403977636e-07,
        "self-harm/instructi

In [17]:
chat_complete_url = f"{base_url}/v1/chat/completions"

def chat_complete(user_msg, system_msg="", model="gpt-4o"):
    data = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": system_msg
            },
            {
                "role": "user",
                "content": user_msg
            }
        ],
        "temperature": 0.5,
        "max_tokens": 256,
        "top_p": 1
    }
    return requests.post(chat_complete_url, headers=headers, json=data).json()['choices'][0]['message']['content']

In [21]:
prompt = "koreans are racist"
mod = get_moderation(prompt)
print(chat_complete(str(mod["results"][0]), system_msg=f"explain why the prompt='{prompt}' was rejected or accepted"))

The prompt "koreans are racist" was rejected because it was flagged for containing hate speech. The evaluation shows high scores in the categories of harassment and hate, indicating that the statement is considered discriminatory and promotes negative stereotypes about a specific ethnic group. Such generalizations are harmful as they unfairly attribute negative traits to an entire group of people based on race or nationality, which can perpetuate prejudice and discrimination. The system is designed to prevent the spread of such harmful content, which is why the prompt was flagged and rejected.


In [22]:
prompt = "chess players are racist"
mod = get_moderation(prompt)
print(chat_complete(str(mod["results"][0]), system_msg=f"explain why the prompt='{prompt}' was rejected or accepted"))

The prompt "chess players are racist" was not flagged, which means it was accepted by the system. The decision to not flag the prompt was likely based on the category scores for potential harmful content. Although the prompt carries a potentially sensitive statement, the scores for categories like "hate" and "harassment" are relatively low, with the highest being "harassment" at 0.3425 and "hate" at 0.1832. These scores indicate that the system did not find the prompt to contain sufficiently high levels of hate speech or harassment to warrant rejection.

Furthermore, none of the categories, including "hate/threatening" and "violence," reached a threshold that would trigger a flag. The system might have determined that the prompt, while provocative, does not explicitly promote hate or violence, nor does it contain any direct threats or harmful instructions.

It's important to note that while the system did not flag the prompt, it doesn't necessarily mean the statement is accurate or app